# ベースラインのモデル訓練

In [1]:
# データ処理ライブラリ
import numpy as np
import pandas as pd 

# 画像処理ライブラリ
import cv2

# 可視化ライブラリ
import matplotlib.pyplot as plt

# Keras
import keras as ks
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Scikit-learn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score

# Pythonモジュールと設定
import os
import glob
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Pathの設定
test_path = "./chest_xray/test/"
train_path = "./chest_xray/train/"

In [3]:
# npyファイルを読み込み
X_train = np.load(train_path + 'X_train.npy')
y_train = np.load(train_path + 'y_train.npy')
X_test = np.load(test_path + 'X_test.npy')
y_test = np.load(test_path + 'y_test.npy')

# サイズの確認
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5216, 64, 64, 3)
(5216,)
(624, 64, 64, 3)
(624,)


In [4]:
# Sequentialインスタンス生成
model = Sequential()

# 入力と畳み込み層

In [5]:
# 入力層と畳み込み層
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(64,64,3)))

Instructions for updating:
Colocations handled automatically by placer.


## プール層とドロップアウト

In [6]:
# プール層の追加
model.add(MaxPooling2D(pool_size=(2, 2)))

# ドロップアウトの追加
model.add(Dropout(0.25))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## 全結合層 x 2（Flatten + Dense）

In [7]:
# 全結合層
model.add(Flatten())
model.add(Dense(128, activation='relu'))

# ドロップアウト
model.add(Dropout(0.33))

## 出力層

In [8]:
model.add(Dense(1, activation='sigmoid'))

## モデルの確認

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30752)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3936384   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total para

## モデルのコンパイル

二項分類のため、損失関数は「binary_crossentropy」を使い、
最適化関数は「Adam」を使います。

「Adam」は Stochastic Gradient Descent（確率的勾配降下法）の拡張版の最適化関数。  
CNNの最適化関数は、「Adam」あるいは「SGD（確率的勾配降下法）」を利用するのが一般的。  

In [10]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.00001),
              metrics=['accuracy'])

## モデルの訓練（Train）

In [11]:
# EarlyStoppingの設定
callbacks = EarlyStopping(monitor='val_loss', patience=5)

In [12]:
# モデル訓練
model.fit(X_train, y_train, validation_data = (X_test, y_test), callbacks=[callbacks], epochs=20)

Instructions for updating:
Use tf.cast instead.
Train on 5216 samples, validate on 624 samples
Epoch 1/20
5216/5216 [==============================] - 15s 3ms/step - loss: 0.5282 - acc: 0.7416 - val_loss: 0.5877 - val_acc: 0.6282
Epoch 2/20
5216/5216 [==============================] - 14s 3ms/step - loss: 0.3935 - acc: 0.8330 - val_loss: 0.4708 - val_acc: 0.7692
Epoch 3/20
5216/5216 [==============================] - 14s 3ms/step - loss: 0.3111 - acc: 0.8890 - val_loss: 0.4611 - val_acc: 0.7516
Epoch 4/20
5216/5216 [==============================] - 14s 3ms/step - loss: 0.2594 - acc: 0.9087 - val_loss: 0.4040 - val_acc: 0.7917
Epoch 5/20
5216/5216 [==============================] - 14s 3ms/step - loss: 0.2282 - acc: 0.9254 - val_loss: 0.4911 - val_acc: 0.7452
Epoch 6/20
5216/5216 [==============================] - 14s 3ms/step - loss: 0.2015 - acc: 0.9302 - val_loss: 0.3622 - val_acc: 0.8205
Epoch 7/20
5216/5216 [==============================] - 14s 3ms/step - loss: 0.1845 - acc: 0.93

## テストデータで推測

In [13]:
# ベースラインモデル テストデータで推測結果出力
base_y_pred = model.predict_classes(X_test)

In [14]:
# 混同行列を算出
confusion_matrix(y_test,base_y_pred, labels=[1, 0])

array([[389,   1],
       [160,  74]], dtype=int64)

In [15]:
# 評価指標 算出
print('正解率：', accuracy_score(y_test, base_y_pred))
print('適合率：', precision_score(y_test, base_y_pred))
print('再現率：', recall_score(y_test, base_y_pred))

正解率： 0.7419871794871795
適合率： 0.7085610200364298
再現率： 0.9974358974358974
